### Prueba 1 - Hito 2 - Javier Rojas Benitez - G56

In [1]:
import importlib

import pandas as pd
import numpy as np
import polars as pl

import matplotlib.pyplot as plt
import seaborn as sns

import warnings

warnings.filterwarnings("ignore")
sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

In [2]:
df = pd.read_csv("input/training_tweets.csv").drop(columns="Unnamed: 0")


# Preprocesamiento

## Variables independientes

In [17]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

STOPWORDS_INGLES = set(stopwords.words("english"))

In [33]:
def preprocesar_texto_lema(texto):
    tokens = texto.split()
    filtro_stop_words = [palabra for palabra in tokens if palabra not in STOPWORDS_INGLES]
    lematizacion = [lemmatizer.lemmatize(palabra, "v") for palabra in filtro_stop_words]

    texto_juntado = " ".join(lematizacion)

    return texto_juntado


In [34]:
df["procesados"] = df.content.str.lower().apply(preprocesar_texto_lema)
